In [13]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Для задачи Question Answering мы используем класс BertForQuestionAnswering из библиотеки transformers.

In [14]:
import pandas as pd
import json
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

Загрузим предъобученную модель BERT

In [15]:
model = BertForQuestionAnswering.from_pretrained('ai-forever/ruBert-base')

Some weights of the model checkpoint at ai-forever/ruBert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the mo

Загрузим и токенайзер

In [16]:
tokenizer = BertTokenizer.from_pretrained('ai-forever/ruBert-base')

# Загрузим датасет

In [ ]:
!wget -nc https://github.com/aspushkarev/nlp-test-task-2023/tree/main/data/raw/train.json

In [23]:
num_questions = 0
num_posible = 0
num_imposible = 0

In [131]:
def get_data(path):
  """
  Below code will only return questions wich have answers (i.e. not the ones flagged as imposible to answer)
  Read file and retrieve the contexts, qustions and answers
  """

  data = pd.read_json(path)

  contexts = []
  questions = []
  answers = []
  num_q = 0
  num_pos = 0
  num_imp = 0

  for text in data['text']:
      num_q += 1
      contexts.append(text.lower())

  for question in data['label']:
      questions.append(question.lower())

  for answer in data['extracted_part']:
      if answer.get('answer_start') == [0]:
          num_imp += 1
      else:
          num_pos += 1
          answers.append(answer.get('text'))

  return num_q, num_pos, num_imp, contexts, questions, answers

In [135]:
num_q, num_pos, num_imp, train_contexts, train_questions, train_answers = get_data('train.json')
num_questions  = num_q
num_posible = num_pos
num_imposible  = num_imp

print(f"Total number of questions: {num_questions}")
print(f"Total number of answerable questions: {num_posible}")
print(f"Total number of impossible questions: {num_imposible}")

Total number of questions: 1799
Total number of answerable questions: 1492
Total number of impossible questions: 307


In [ ]:
# data = pd.read_json("train.json")
# data.tail(2)

In [125]:
# with open('train.json', 'rb') as f:
#     raw_data = json.load(f)

# contexts = []
# questions = []
# answers = []
# num_q = 0
# num_pos = 0
# num_imp = 0

# for text in data['text']:
#     num_q += 1
#     contexts.append(text.lower())

# for question in data['label']:
#     questions.append(question.lower())

# for answer in data['extracted_part']:
#     if answer.get('answer_start') == [0]:
#         num_imp += 1
#     else:
#         num_pos += 1
#         answers.append(answer.get('text'))

In [109]:
# print(contexts[:3])
# print(len(contexts))
# print(num_q)

['извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 общая информация номер извещения 0328300032822000806 наименование объекта закупки поставка продуктов питания способ определения поставщика (подрядчика, исполнителя) открытый конкурс в бль порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 федерального закона № 44-фз. выбор способа обеспечения осуществляется участником закупки самостоятельно. срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 федерального закона № 44-фз. условия независимой гарантии в соответствии со ст. 45 федерального закона № 44-фз. реквизиты счета в соответствии с п.16 ч. 

In [113]:
# print(questions[:3])
# print(len(questions))
# print(num_pos)
# print(num_imp)

['обеспечение исполнения контракта', 'обеспечение исполнения контракта', 'обеспечение исполнения контракта']
1799
1492
307


In [116]:
# num_q == num_pos + num_imp

True

In [132]:
print(train_questions[0:3])
print(train_answers[0:3])

['обеспечение исполнения контракта', 'обеспечение исполнения контракта', 'обеспечение исполнения контракта']
[['Размер обеспечения исполнения контракта 6593.25 Российский рубль'], ['Поставщик должен предоставить обеспечение исполнения контракта в размере 10 % от цены Контракта.'], ['Размер обеспечения исполнения контракта 10.00%']]


In [136]:
def add_answer_end(answers, contexts):
  for answer, context in zip(answers, contexts):
    answer['text'] = answer['text'].lower()
    answer['answer_end'] = answer['answer_start'] + len(answer['text'])

add_answer_end(train_answers, train_contexts)
# add_answer_end(valid_answers, valid_contexts)

TypeError: ignored

In [ ]:
tokenizerFast = BertTokenizerFast.from_pretrained(MODEL_PATH)

train_encodings_fast = tokenizerFast(train_questions, train_contexts,  max_length = MAX_LENGTH, truncation=True)